#Problem liniowy
Instalacja biblioteki

In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


In [2]:
### Kod sprawdza, czy instalacja się udała
import pulp
print(pulp.__version__)

2.4


In [ ]:
pulp.pulpTestAll()

# Zadania:

1. Przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. W blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %



In [4]:
# Import z przestrzeni nazw
from pulp import *

##Wersja nieuporządkowana, uproszczona do 3 składników:

In [5]:
# Deklaracja problemu
kosztPuszki = LpProblem("Whiskas_Problem", LpMinimize)
# Zmienne z limitem 0-100%
zwrtKurcz = LpVariable("Chicken_Percent", 0, 100, LpInteger)
zwrtWolow = LpVariable("Beef_Percent", 0, 100, LpInteger)
zwrtRyzu  = LpVariable("Rice_Percent", 0, 100, LpInteger)
# Dodanie funkcji określającej problem
kosztPuszki += 0.013 * zwrtKurcz + 0.008 * zwrtWolow + 0.007 * zwrtRyzu, "Ingredients_Cost"
# Ograniczenia składu
kosztPuszki += zwrtKurcz + zwrtWolow + zwrtRyzu == 100, "Total_Ingredients_Percent"
kosztPuszki += 0.100 * zwrtKurcz + 0.200 * zwrtWolow + 0.000 * zwrtRyzu >= 8.0, "Protein_Content"
kosztPuszki += 0.080 * zwrtKurcz + 0.100 * zwrtWolow + 0.010 * zwrtRyzu >= 6.0, "Fat_Content"
kosztPuszki += 0.001 * zwrtKurcz + 0.005 * zwrtWolow + 0.100 * zwrtRyzu <= 2.0, "Fiber_Content"
kosztPuszki += 0.002 * zwrtKurcz + 0.005 * zwrtWolow + 0.002 * zwrtRyzu <= 0.4, "Salt_Content"
# Zawartość witaminy B w ug
kosztPuszki += 0.003 * zwrtKurcz + 0.026 * zwrtWolow + 0.000 * zwrtRyzu >= 0.8, "VitaminB_Content"

# Tworzenie pliku w odpowiednim języku
kosztPuszki.writeLP("WhiskasModel.lp")
kosztPuszki.solve()

# Status rozwiązania
print("Status:", LpStatus[kosztPuszki.status])
# Wykorzystanie składników
for zm in kosztPuszki.variables():
    print(zm.name, "=", zm.varValue, "%")
# Podsumowanie
print("One can (100g) costs: {koszt} $. Ingredients: {sklad}".format(
    koszt = kosztPuszki.objective.value(),
    sklad = ", ".join(skl.name[0:skl.name.index('_')] + " (" + str(skl.varValue) + "%)"
    for skl in kosztPuszki.variables()))
)


Status: Optimal
Beef_Percent = 66.0 %
Chicken_Percent = 18.0 %
Rice_Percent = 16.0 %
One can (100g) costs: 0.874 $. Ingredients: Beef (66.0%), Chicken (18.0%), Rice (16.0%)


##Wersja z użyciem słowników
W składzie ma się pojawić dodany produkt (sardynki - bardzo drogie), a więc konieczna jest modyfikacja założeń.

###Promocja na sardynki
W tym przypadku udało się znaleźć okazyjnie tanie sardynki, które są bardziej wartościowe od wołowiny, a kosztują tyle samo (więcej wartości odżywczych w tej samej cenie). Można więc kupić ich mniej, a resztę wypełnić tanim żelem i zmniejszyć koszt produktu.

In [6]:
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.0015,
         'SARDINES': 0.008}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'SARDINES': 0.250}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.110}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES': 0.003}
              
vitaminBPercent = {'CHICKEN': 0.003, 
               'BEEF':        0.026, 
               'MUTTON':      0.026, 
               'RICE':        0.000, 
               'WHEAT':       0.000, 
               'GEL':         0.000,
               'SARDINES':    0.089}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The_Whiskas_Problem", LpMinimize)

# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr", Ingredients, 0, 100)

# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

# The constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([vitaminBPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.8, "VitBRequirement"

prob.writeLP("WhiskasModelBis.lp")
prob.solve()
print("Status: ", LpStatus[prob.status])

# Podsumowanie
print("One can (100g) costs: {cost:.3f}$. Ingredients: {ingrList}".format(
    cost = prob.objective.value(),
    ingrList = ", ".join(
        "{} ({:.2f}%)".format(i.name[i.name.index('_') + 1:].capitalize(), i.varValue)
        for i in prob.variables() if i.varValue > 0)
))

Status:  Optimal
One can (100g) costs: 0.503$. Ingredients: Gel (27.27%), Rice (20.00%), Sardines (52.73%)


###Terapia witaminowa
Załóżmy, że kot senior potrzebuje więcej witaminy B. Nawet mimo, że sardynki są droższe od wołowiny, to zawierają potrzebną witaminę (więcej, więc z punktu widzenia tego mikroelementu taniej). Są więc składnikiem uzupełniającym.

In [7]:
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.0015,
         'SARDINES': 0.010}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'SARDINES': 0.250}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.110}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES': 0.003}
              
vitaminBPercent = {'CHICKEN': 0.003, 
               'BEEF':        0.026, 
               'MUTTON':      0.026, 
               'RICE':        0.000, 
               'WHEAT':       0.000, 
               'GEL':         0.000,
               'SARDINES':    0.089}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The_Whiskas_Problem", LpMinimize)

# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr", Ingredients, 0, 100)

# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

# The constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([vitaminBPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 1.6, "VitBRequirement"

prob.writeLP("WhiskasModelBis.lp")
prob.solve()
print("Status: ", LpStatus[prob.status])

# Podsumowanie
print("One can (100g) costs: {cost:.3f}$. Ingredients: {ingrList}".format(
    cost = prob.objective.value(),
    ingrList = ", ".join(
        "{} ({:.2f}%)".format(i.name[i.name.index('_') + 1:].capitalize(), i.varValue)
        for i in prob.variables() if i.varValue > 0)
))

Status:  Optimal
One can (100g) costs: 0.539$. Ingredients: Beef (56.74%), Gel (24.69%), Rice (17.16%), Sardines (1.40%)


##Pliki z modelami problemów

In [ ]:
!cat "WhiskasModel.lp"

In [ ]:
!cat "WhiskasModelBis.lp"

#Podsumowanie
Użycie słowników umożliwia definiowanie bardziej rozbudowanych problemów bez znacznego zwiększania nakładu pracy. Dodatkowo, problem opisany tą metodą jest znacznie bardziej przejrzysty, dzięki czemu można łatwo wprowadzać modyfikacje założeń. Dane do takiego zadania mogłyby być również zdefiniowane w zewnętrznych plikach, gdzie tworzenie oddzielnych zmiennych na każdą wartość byłoby zdecydowanie niewskazane.